In [22]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [21]:
!echo $CUDA_VISIBLE_DEVICES

1


In [3]:
# tf.config.set_visible_devices(tf.config.list_physical_devices()[0])

In [4]:
import sys, os, time
from pathlib import Path
from glob import glob

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import cv2
import tensorflow as tf

CWD = Path.cwd()
os.chdir(str(CWD.parent.parent))
import utils as ut
os.chdir(str(CWD))

%matplotlib inline

In [5]:
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

## Determine paths and directories

In [6]:
path_imagenet_val_dataset = Path("/home/97213133/Datasets/ImageNet/Validation/") # path/to/data/
dir_images = path_imagenet_val_dataset / "images" # path/to/images/directory
path_labels = path_imagenet_val_dataset / "ILSVRC2012_validation_ground_truth.txt"
path_synset_words = path_imagenet_val_dataset / "synset_words.txt"
path_meta = path_imagenet_val_dataset / "meta.mat"
Path_generated_labels = path_imagenet_val_dataset / "generated_labels.txt"

In [7]:
image_paths = sorted(glob(str(dir_images/"*")))
n_images = len(image_paths)

n_images

50000

## Preprocess images

In [17]:
def load_images(image_paths, returned_shard, n_shards=5):
    """ loads images into memory. It only load and returns images of the 'returned_shard'.
        image_paths: a list of paths to images
        n_shards: number of shards to loaded images be divided.
        returned_shard: the part of images to be returned. 0 <= returned_shard < n_shards
    """
    assert 0 <= returned_shard < n_shards, "The argument returned_shard must be between 0 and n_shards"
    
    shard_size = len(image_paths) // n_shards
    sharded_image_paths = image_paths[returned_shard*shard_size:(returned_shard+1)*shard_size] if returned_shard < n_shards - 1 \
                     else image_paths[returned_shard*shard_size:]
    
    images_list = np.zeros((len(sharded_image_paths), 224, 224, 3), dtype=np.uint8)
    
    for i, image_path in enumerate(sharded_image_paths):
        
        # Load (in BGR channel order)
        image = cv2.imread(image_path)
        
        
        # Resize
        height, width, _ = image.shape
        new_height = height * 256 // min(image.shape[:2])
        new_width = width * 256 // min(image.shape[:2])
        image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

        # Crop
        height, width, _ = image.shape
        startx = width//2 - (224//2)
        starty = height//2 - (224//2)
        image = image[starty:starty+224,startx:startx+224]
        assert image.shape[0] == 224 and image.shape[1] == 224, (image.shape, height, width)
        

        images_list[i, ...] = image[..., ::-1]
        
    return images_list

In [8]:
@tf.function
def load_images(image_paths, returned_shard, n_shards=5):
    """ loads images into memory. It only load and returns images of the 'returned_shard'.
        image_paths: a list of paths to images
        n_shards: number of shards to loaded images be divided.
        returned_shard: the part of images to be returned. 0 <= returned_shard < n_shards
    """
    assert 0 <= returned_shard < n_shards, "The argument returned_shard must be between 0 and n_shards"
    
    shard_size = len(image_paths) // n_shards
    sharded_image_paths = image_paths[returned_shard*shard_size:(returned_shard+1)*shard_size] if returned_shard < n_shards - 1 \
                     else image_paths[returned_shard*shard_size:]
    
#     images_list = tf.zeros((len(sharded_image_paths), 224, 224, 3), dtype=np.uint8)
    images_list = []
    
    for i, image_path in enumerate(sharded_image_paths):
        
        # Load (in BGR channel order)
        image = cv2.imread(image_path)
        
        new_height, new_width = np.array(image.shape[:2]) * 256 // np.min(image.shape[:2])
        
        image = tf.image.resize(image, (new_height, new_width), method='bicubic')
        
        start_y, start_x = np.array(image.shape[:2]) // 2 - (224 // 2)
        image = image[start_y:start_y + 224, start_x:start_x + 224]
        assert image.shape[0] == 224 and image.shape[1] == 224

        images_list.append(image[..., ::-1])
        
    return tf.stack(images_list)

In [25]:
def load_images(image_paths, returned_shard, n_shards=5):
    """ loads images into memory. It only load and returns images of the 'returned_shard'.
        image_paths: a list of paths to images
        n_shards: number of shards to loaded images be divided.
        returned_shard: the part of images to be returned. 0 <= returned_shard < n_shards
    """
    assert 0 <= returned_shard < n_shards, "The argument returned_shard must be between 0 and n_shards"
    
    shard_size = len(image_paths) // n_shards
    sharded_image_paths = image_paths[returned_shard*shard_size:(returned_shard+1)*shard_size] if returned_shard < n_shards - 1 \
                     else image_paths[returned_shard*shard_size:]
    
#     images_list = tf.zeros((len(sharded_image_paths), 224, 224, 3), dtype=np.uint8)
    images_list = []
    
    for i, image_path in enumerate(sharded_image_paths):
        
        # Load (in BGR channel order)
        image = cv2.imread(image_path)
        
        # Resize
        new_height, new_width = np.array(image.shape[:2]) * 256 // np.min(image.shape[:2])
#         print("salam")
        image = tf.image.resize(image, (new_height, new_width), method='bicubic')
#         print("kkkkkk")
        # Crop
#         image = tf.image.central_crop(image, .875)
        
#         image = tf.cast(tf.image.resize(image, (224, 224), method='lanczos5'), tf.uint8)
        
#         start_y, start_x = np.array(image.shape[:2]) // 2 - (224 // 2)
#         image = image[start_y:start_y + 224, start_x:start_x + 224]
#         assert image.shape[0] == 224 and image.shape[1] == 224
#         print("DDDDD")
        # Convert image color channels
#         images_list[i,:,:,:] = image[:,:,::-1]
#         images_list.append(image.numpy()[..., ::-1])
        images_list.append(image[..., ::-1])
        
    return tf.stack(images_list)

In [ ]:
def load_images(image_paths, returned_shard, n_shards=5):
    """ loads images into memory. It only load and returns images of the 'returned_shard'.
        image_paths: a list of paths to images
        n_shards: number of shards to loaded images be divided.
        returned_shard: the part of images to be returned. 0 <= returned_shard < n_shards
    """
    assert 0 <= returned_shard < n_shards, "The argument returned_shard must be between 0 and n_shards"
    
    shard_size = len(image_paths) // n_shards
    sharded_image_paths = image_paths[returned_shard*shard_size:(returned_shard+1)*shard_size] if returned_shard < n_shards - 1 \
                     else image_paths[returned_shard*shard_size:]
    
    images_list = np.zeros((len(sharded_image_paths), 224, 224, 3), dtype=np.uint8)
    
    for i, image_path in enumerate(sharded_image_paths):
        
        # Load (in BGR channel order)
        image = cv2.imread(image_path)
        
        # Resize
        new_height, new_width = np.array(image.shape[:2]) * 256 // np.min(image.shape[:2])      
        image = cv2.resize(image, (new_height, new_width), interpolation=cv2.INTER_CUBIC)
        
        # Crop
        start_y, start_x = np.array(image.shape[:2]) // 2 - (224 // 2)
        image = image[start_y:start_y + 224, start_x:start_x + 224]
        assert image.shape[0] == 224 and image.shape[1] == 224
        
        # Convert image color channels
        images_list[i,:,:,:] = image[:,:,::-1]
        
    return images_list

## Load and save images in 50 shards

In [23]:
ut.check_create_path(str(path_imagenet_val_dataset / 'shards'))
n_shards = 500

for i in range(n_shards):
    
    images = load_images(image_paths, returned_shard=i, n_shards=n_shards)
    
    if i == 0:
        print("Total memory allocated for loading images:", ut.humansize(images.nbytes))
    
    np.save(str(path_imagenet_val_dataset / "x_val_{}.npy".format(i+1)), images)
#     tf.io.write_file(str(path_imagenet_val_dataset / "x_val_{}.tensor".format(i+1)), images)
    
    if (i + 1) * 100 / n_shards % 5 == 0:
        print("{:.0f}% Completed.".format((i+1)/n_shards*100))
    
    images = None

Total memory allocated for loading images: 14.36 MB
5% Completed.
10% Completed.
15% Completed.
20% Completed.
25% Completed.
30% Completed.
35% Completed.
40% Completed.
45% Completed.
50% Completed.
55% Completed.
60% Completed.
65% Completed.
70% Completed.
75% Completed.
80% Completed.
85% Completed.
90% Completed.
95% Completed.
100% Completed.


## Mapping between different label types

In [11]:
meta = scipy.io.loadmat(str(path_meta))
original_idx_to_synset = {}
synset_to_name = {}

for i in range(1000):
    ilsvrc2012_id = int(meta["synsets"][i,0][0][0][0])
    synset = meta["synsets"][i,0][1][0]
    name = meta["synsets"][i,0][2][0]
    original_idx_to_synset[ilsvrc2012_id] = synset
    synset_to_name[synset] = name

synset_to_keras_idx = {}
keras_idx_to_name = {}
with open(str(path_synset_words), "r") as f:
    for idx, line in enumerate(f):
        parts = line.split(" ")
        synset_to_keras_idx[parts[0]] = idx
        keras_idx_to_name[idx] = " ".join(parts[1:])

convert_original_idx_to_keras_idx = lambda idx: synset_to_keras_idx[original_idx_to_synset[idx]]

In [30]:
original_idx_to_synset

{1: 'n02119789',
 2: 'n02100735',
 3: 'n02110185',
 4: 'n02096294',
 5: 'n02102040',
 6: 'n02066245',
 7: 'n02509815',
 8: 'n02124075',
 9: 'n02417914',
 10: 'n02123394',
 11: 'n02125311',
 12: 'n02423022',
 13: 'n02346627',
 14: 'n02077923',
 15: 'n02110063',
 16: 'n02447366',
 17: 'n02109047',
 18: 'n02089867',
 19: 'n02102177',
 20: 'n02091134',
 21: 'n02092002',
 22: 'n02071294',
 23: 'n02442845',
 24: 'n02504458',
 25: 'n02092339',
 26: 'n02098105',
 27: 'n02096437',
 28: 'n02114712',
 29: 'n02105641',
 30: 'n02128925',
 31: 'n02091635',
 32: 'n02088466',
 33: 'n02096051',
 34: 'n02117135',
 35: 'n02138441',
 36: 'n02097130',
 37: 'n02493509',
 38: 'n02457408',
 39: 'n02389026',
 40: 'n02443484',
 41: 'n02110341',
 42: 'n02089078',
 43: 'n02086910',
 44: 'n02445715',
 45: 'n02093256',
 46: 'n02113978',
 47: 'n02106382',
 48: 'n02441942',
 49: 'n02113712',
 50: 'n02113186',
 51: 'n02105162',
 52: 'n02415577',
 53: 'n02356798',
 54: 'n02488702',
 55: 'n02123159',
 56: 'n02098413',
 

In [12]:
with open(str(path_labels),"r") as f:
    y_val = f.read().strip().split("\n")
    y_val = np.array([convert_original_idx_to_keras_idx(int(idx)) for idx in y_val])

In [ ]:
np.save(str(path_imagenet_val_dataset/"y_val.npy"), y_val)

## Show example images

### Load a shard of validation images

In [ ]:
idx_shard = 4
x_val = np.load(str(path_imagenet_val_dataset/"x_val_{}.npy").format(idx_shard))

### Show a few images

In [ ]:
n_images2show = 15
n_cols = 3
n_rows = 15 // n_cols
figsize = (20, 20)

indices = np.random.choice(x_val.shape[0], size=n_images2show, replace=False)
images = x_val[indices] / 255.

fig, ax = plt.subplots(figsize=figsize, nrows=n_rows, ncols=n_cols)
for i, axi in enumerate(ax.flat):
    
    axi.imshow(images[i])
    label_index = (idx_shard - 1) * (n_images // n_shards) + indices[i]
    axi.set_title(keras_idx_to_name[y_val[label_index]], y=.9, fontdict={'fontweight':'bold'}, pad=10)
    axi.set_axis_off()

plt.show()